<a href="https://colab.research.google.com/github/levidasilvalima/RNA-PP1/blob/master/RedesNeuraisArtificiais_Projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTAÇÕES

from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# CRIAÇÃO DO DRIVE

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
# Importar o arquivo csv
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Manaus.csv', sep=';', encoding='ISO-8859-1')

NameError: ignored

In [3]:
# Teste do DataSet CSV importado
df

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')